In [3]:
from Helper.Load.load import Base_Load
from Helper.Load.supervised import Supervised
from Helper.Load.regression import Regression
from Helper.Load.classification import Classification
from Helper.Visual.Visual import Visual
from Helper.Visual.classification import VClassification
from Helper.Transform.Transform.classification import Classificaton_Transform
from Helper.Train.Train import Train

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
loadC = Classification("Data", "led7digit.dat", "led7digit-names.txt")
#transformC = Classificaton_Transform(loadC)
#trainC = Train(loadC, Classificaton_Transform, RandomForestClassifier)

In [8]:
loadC.partition.X_train

,Led1,Led2,Led3,Led4,Led5,Led6,Led7
232,1.0,0.0,1.0,1.0,0.0,1.0,1.0
498,1.0,1.0,1.0,0.0,0.0,1.0,0.0
37,0.0,1.0,1.0,1.0,1.0,1.0,1.0
163,1.0,0.0,1.0,0.0,0.0,1.0,0.0
83,1.0,1.0,1.0,0.0,1.0,1.0,1.0
339,1.0,1.0,0.0,1.0,1.0,1.0,1.0
230,1.0,0.0,1.0,1.0,0.0,0.0,1.0
43,1.0,1.0,1.0,1.0,0.0,1.0,1.0
415,0.0,0.0,1.0,1.0,0.0,1.0,1.0
101,0.0,1.0,1.0,1.0,0.0,1.0,0.0
